## Web scrapping de IMDB

Descarga la información correspondiente y guarda en un csv el top de las 250 películas mediante webscrapping. Encapsúlalo en un script.

Obtén:
* Título
* Año
* Duración
* Posición
* Rating

In [ ]:
# Si la petición te devuelve un 403, puedes probar con:
# pip install fake-useragent
# from fake_useragent import UserAgent
# ua = UserAgent()
# headers = {'User-Agent': ua.random}
# response = requests.get(url, headers=headers)

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

url = "https://www.imdb.com/chart/top/"
headers = {"User-Agent": "Mozilla/5.0"}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

script_tag = soup.find("script", id="__NEXT_DATA__")
data = json.loads(script_tag.string)
data


In [ ]:

titles = data["props"]["pageProps"]["pageData"]["chartTitles"]["edges"]

peliculas = {
    "titulo": [],
    "anio": [],
    "duracion": [],
    "posicion": [],
    "rating": []
}

for movie in titles:
    node = movie["node"]

    titulo = node.get("originalTitleText", {}).get("text") or node.get("titleText", {}).get("text")
    anio = node.get("releaseYear", {}).get("year", "N/A")
    duracion_seg = node.get("runtime", {}).get("seconds", 0)
    duracion_min = round(duracion_seg / 60) if duracion_seg else "N/A"
    posicion = movie.get("currentRank", "N/A")
    rating = node.get("ratingsSummary", {}).get("aggregateRating", "N/A")

    peliculas["titulo"].append(titulo)
    peliculas["anio"].append(anio)
    peliculas["duracion"].append(duracion_min)
    peliculas["posicion"].append(posicion)
    peliculas["rating"].append(rating)

df = pd.DataFrame(peliculas)
df.to_csv("peliculas_imdb.csv")


df.head()